In [4]:
import requests
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
es_client = Elasticsearch('http://localhost:9200')

index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'
es_client.indices.create(index=index_name, body=index_setting)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [7]:
index_name = 'course-questions'
query = "How do I execute a command in a running docker container?"
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [9]:
response['hits']['hits'][0]['_score']

84.050095

In [14]:
response['hits']['hits'][-1]

{'_index': 'course-questions',
 '_id': 'pp_QWpAB4oStZdWMEwOF',
 '_score': 49.938507,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

In [10]:
result_docs = []
    
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [11]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context += """
Q: {question}
A: {text}
""".format(question=doc['question'], text=doc['text']).strip()
        context += '\n\n'

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [12]:
prompt = build_prompt(query, result_docs)

In [13]:
len(prompt)

1462

In [15]:
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [16]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [17]:
len(encoding.encode(prompt))

322